### ESCOM - IIA
#### FUNDAMENTOS DE INTELIGENCIA ARTIFICIAL - PROYECTO ML
#### Semestre 2023-2 - Grupo 4BV2
--- 
##### Autores:
- **Valdés Luis Eliot Fabián**

In [ ]:
# importamos la librerias necesarias
from ucimlrepo import fetch_ucirepo  # libreria de repositorios de datasets
import pandas as pd  # libreria para el manejo de dataframes

----
# PASO 1: Carga de datos
Sección inicial para obtener el dataset, describirlo de manera general usando estadisticas

In [ ]:
# definimos los datasets a utilizar
datasets = {
    "Eliot": 53, # Iris
    "Ethel": 728, # 
    "Leo": 878, # Cirrhosis
    "Adair": 109, # Wine
}
dataset = fetch_ucirepo(id=datasets["Leo"]) # cargamos el dataset que deseemos utilizar

In [ ]:
# mostramos un pequeño resumen de lo que trata el dataset 
dataset.metadata.abstract

In [ ]:
# obtenemos los datos
X = dataset.data.features 
y = dataset.data.targets 

In [ ]:
# creamos un dataframe con los datos
df = pd.DataFrame(X, columns=dataset.data.feature_names)
# agregamos la columna target
df['target'] = y

# mostramos la cantidad de registros
print(f'Cantidad de registros: {len(df)}')

# imprimimos los primeros 5 registros
df.head()

In [ ]:
# mostramos los tipos de datos de cada columna
print(f"Tipos de datos (Metodo pandas):\n{df.dtypes}\n")

In [ ]:
# mostramos los tipos de datos usando los metodos de la librearia de repositorios
dataframe_tipos = pd.DataFrame({'Variable Name': dataset.variables['name'], 'Type': dataset.variables['type']})
print(f"Tipos de datos (Metodo ucimlrepo):\n{dataframe_tipos}\n")

In [ ]:
# por cada columna hacer lo siguiente 
# si el tipo de dato es numerico, mostrar la media, mediana, desviacion estandar, minimo y maximo
# si el tipo de dato es categorico, mostrar la cantidad de valores unicos

# limitamos la cantidad de columnas a 10
for col in df.columns[:10]:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        print(f"=> '{col}':")
        print(f"\t-Media: {df[col].mean()}")
        print(f"\t-Mediana: {df[col].median()}")
        print(f"\t-Desviacion estandar: {df[col].std()}")
        print(f"\t-Minimo: {df[col].min()}")
        print(f"\t-Maximo: {df[col].max()}")
    else:
        print(f"=> '{col}':")
        print(f"\t-Valores unicos: {df[col].unique()}")
        print(f"\t-Cantidad de valores unicos: {df[col].nunique()}")

----

# PASO 2: PREPROCESAMIENTO DE DATOS

Sección para preprocesar los datos, aquí separamos el dataset en los vectores de entrada X & de salida Y(las clases que son definidas con tipo de dato categorico)


In [ ]:
# funcion para separar cualquier dataframe en dos vectores (entrada y salida)
def separate_dataframe(df):
    vector_x = df.drop(df.columns[-1], axis=1) # obtencion de variables de entrada
    vector_y = df[df.columns[-1]] # obtencion de variable de salida (clase/target)
    return vector_x, vector_y

# separamos el dataframe en dos vectores, uno con las variables independientes y otro con la variable dependiente
vector_x, vector_y = separate_dataframe(df)

In [ ]:
# mostramos los primeros 5 registros del vector de entrada X
vector_x.head()

In [ ]:
# mostramos los primeros 5 registros del vector de salida Y
vector_y.head()

In [ ]:
# creamos una funcion que recibe como parametros el vector de entrada X y el vector de salida Y
def describe_categories(vector_x, vector_y):
    # agrupamos el vector de entrada X de acuerdo al vector de salida Y
    grouped = vector_x.groupby(vector_y)
    for name, group in grouped:
        print("\n",("="*50))
        print(f'Clase: {name}')
        print("="*50)
        # por cada grupo, accedemos a cada una de las columnas
        for col in group.columns:
            # validamos que el tipo de dato de la columna sea numerico y de ser el caso mosstramos las estadisticas
            if group[col].dtype == 'float64' or group[col].dtype == 'int64':
                print(f"=> '{col}':")
                print(f"\t-Media: {group[col].mean()}")
                print(f"\t-Mediana: {group[col].median()}")
                print(f"\t-Desviacion estandar: {group[col].std()}")
                print(f"\t-Minimo: {group[col].min()}")
                print(f"\t-Maximo: {group[col].max()}")
            else:
                print(f"=> '{col}':")
                print(f"\tValores unicos: {group[col].unique()}")
                print(f"\tCantidad de valores unicos: {group[col].nunique()}")            
        

describe_categories(vector_x, vector_y)

-----
# PASO 3: LIMPIEZA DE DATASET
Sección para limpiar el dataset para eliminar valores que no aportan al modelo, como los valores nulos o NaN.

In [ ]:
# juntamos los vectores de entrada y salida en un solo dataframe
df = vector_x.join(vector_y)
# mostramos la cantidad de registros
print(f'Cantidad de registros (Before cleaning): {len(df)}')

In [ ]:
# eliminamos los registros que tengan valores nulos en alguna de las columnas
df = df.dropna()
# mostramos la cantidad de registros
print(f'Cantidad de registros: (After cleaning) {len(df)}')

In [ ]:
# de nuevo separamos el dataframe en dos vectores, uno con las variables independientes y otro con la variable dependiente
vector_x, vector_y = separate_dataframe(df)

In [ ]:
# conservamos unicamente las columnas con tipo de dato Inter o Continuous
vector_x = vector_x.select_dtypes(include=['int64', 'float64'])
# mostramos las primeras 5 filas del vector de entrada
vector_x.head()

In [ ]:
print(f'Otra forma de limpiar el vector de entrada. Usa el dataset original y los tipos de datos del dataset pero no es tan eficiente como el metodo anterior ya que alginas columnas con tipo de dato numerico no son numericas en el dataset')
"""
for col_name in vector_x.columns:
    # verificar el tipo de dato de cada columna usando el dataset
    if col_name in dataset.variables['name'].values:
        # obtener el tipo de dato de la columna en el dataset
        col_type = dataset.variables['type'][dataset.variables['name'] == col_name].values[0]
        # eliminamos del vector_x aquellas columnas que no son numericas
        if col_type != 'Integer' and col_type != 'Continuous':
            vector_x = vector_x.drop(col_name, axis=1)
        else:        
            # parseamos las columnas a tipo de dato float64
            vector_x[col_name] = vector_x[col_name].astype('float64')                    

# mostramos las primeras 5 filas del vector de entrada
vector_x.head()                    
"""

-----
# PASO 4: IMPLEMENTACIÓN DE MODELOS DE ML Y EVALUACIÓN CON DIFERENTES MÉTRICAS
Sección para implementar los modelos Minima Distancia y KNN (K=1) para evaluar con los metodos: train-test split, k-fold cross validation y bootstrapping.